### Instalar la libreria para ejecutar lineas de comando de AWS

In [1]:
!pip install awscli -quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9.1
    Uninstalling rsa-4.9.1:
      Successfully uninstalled rsa-4.9.1
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.2.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 which is incompatible.


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Conectar el Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_folder = "/content/drive/MyDrive/00 VIU/10 TFM/Datasets/CAMELYON17/"
images_folder = base_folder + "images/"

Mounted at /content/drive


In [5]:
# leer de AWS el archivo plano con la información de etiquetas
!aws s3 cp --no-sign-request s3://camelyon-dataset/CAMELYON17/stages.csv "stages.csv"

download: s3://camelyon-dataset/CAMELYON17/stages.csv to ./stages.csv


In [7]:
df_labels = pd.read_csv("stages.csv")
df_labels.head()

,patient,stage,center
0,patient_000.zip,pN0,0
1,patient_000_node_0.tif,negative,0
2,patient_000_node_1.tif,negative,0
3,patient_000_node_2.tif,negative,0
4,patient_000_node_3.tif,negative,0


In [15]:
# Crear un campo con el valor binario de la etiqueta
positivo = ['macro','micro','pN1','pN1mi','pN2'] #lista de stage que se consideran "positivo"
df_labels['etiqueta'] = df_labels['stage'].apply(lambda x: 1 if x in positivo else 0)

In [22]:
# Adicionar un campo con el id del paciente:
df_labels['id_paciente'] = df_labels['patient'].str[8:11]

In [ ]:
# Adicionar un campo con la etiqueta del paciente


In [24]:
df_etiquetas = df_labels[['id_paciente', 'etiqueta']].drop_duplicates()
print(len(df_etiquetas))


165


In [27]:
# Crear la etiqueta de cada paciente (+.zip)
condiciones = [(df_labels['stage'].isin(positivo)) & (df_labels['patient'].str.contains("zip")),
    (~df_labels['stage'].isin(positivo)) & (df_labels['patient'].str.contains("zip"))]
valores  = ['1', '0']
df_labels['etiqueta_paciente'] = np.select(condiciones, valores, default='NA')
display(df_labels.head())

,patient,stage,center,etiqueta,id_paciente,new_etiqueta,etiqueta_paciente
0,patient_000.zip,pN0,0,0,000,0,0
1,patient_000_node_0.tif,negative,0,0,000,NA,NA
2,patient_000_node_1.tif,negative,0,0,000,NA,NA
3,patient_000_node_2.tif,negative,0,0,000,NA,NA
4,patient_000_node_3.tif,negative,0,0,000,NA,NA


In [29]:
e0  = df_labels[df_labels['etiqueta_paciente'] == '0'].shape[0]
print(f"Pacientes con etiqueta cero: {e0}")
e1 = df_labels[df_labels['etiqueta_paciente'] == '1'].shape[0]
print(f"Pacientes con etiqueta uno: {e1}")

Pacientes con etiqueta cero: 35
Pacientes con etiqueta uno: 65
